# Create Custom Genres

A central result of our research is the 'social genres' we were able to create. For this we used the Reddit embedding alongside a hierarchical model. 

In [1]:
import pandas as pd 
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
import os
os.chdir(r'C:\Users\DELL XPS\Desktop\reddit_music')

metadata=pd.read_table('data\\reddit-master-metadata.tsv')
vectors=pd.read_table("data\\reddit-master-vectors.tsv",names=list(range(150)))
metadata=metadata['community']


data=pd.merge(metadata,vectors,left_index=True,right_index=True)

df = pd.read_csv("data\\Reddit_Sub_Com_v4_clean.csv")
df = df.drop_duplicates(subset=['id'])
artist_counts = df.groupby('artist')['score'].count()
artist_counts = artist_counts.reset_index()
artist_counts.columns = ['artist','count']
artist_counts.to_csv("data\\artist_counts.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
def create_genres(df, n_clusters=50,with_ppmi=False):
    """df should be subreddit artist and ppmi. Columns should be ['subreddit','artist','ppmi'] """

    global data 
    if with_ppmi==False:
        df=df[['subreddit','artist']]
        
        merged=pd.merge(df,data,left_on='subreddit',right_on='community')
        if len(merged) != len(df):
            print("we may have a problem")

        count=merged.groupby('artist')['artist'].count()
        artist=merged.groupby('artist')[list(range(150))].mean()

        #only take artists with 20 or more shares 
        a_index=count[count>20].index
        artist=artist.loc[artist.index.isin(a_index)]

        # use hierarchical clustering
        ac=AgglomerativeClustering(n_clusters)

        ac.fit(artist)
        artist['label']=ac.labels_
        return artist, ac 
    elif with_ppmi==True:
        df=df[['subreddit','artist', 'ppmi']]
        merged=pd.merge(df,data,left_on='subreddit',right_on='community')
        for j in range(150):
            merged[j]=merged['ppmi']*merged[j]
        count=merged.groupby('artist')['artist'].count()
        artist=merged.groupby('artist')[list(range(150))].mean()

        #only take artists with 20 or more shares 
        a_index=count[count>20].index
        artist=artist.loc[artist.index.isin(a_index)]

        # use hierarchical clustering
        ac=AgglomerativeClustering(n_clusters)

        ac.fit(artist)
        artist['label']=ac.labels_
        return artist, ac


In [38]:
custom, ac = create_genres(df, 50, with_ppmi=False)
custom=custom.reset_index()


In [39]:
custom.to_csv("data\\Artist_Vectors_f.csv")


custom[['artist','label']].to_csv("data\\Artist_Genre_f.csv")

custom.groupby('label')[list(range(150))].mean().to_csv("Genre_Vectors_f.csv")


In [40]:
# save the model
from joblib import dump, load
dump(ac, 'data\\agglogomerative_f.joblib') 

['data\\agglogomerative_f.joblib']

In [41]:
# save the new finalized dataset
data4=pd.merge(df,custom[['artist','label']], on = 'artist')
data4 = data4[['score','link_id','author','subreddit','created_utc','id','parent_id','track_id','artist','track_name','link','youtube_id','class','url','album_id','label']]
data4.to_csv("data\\Reddit_Sub_Com_f_genre.csv", index=False)


In [45]:
data4.groupby('artist')['artist'].count().sort_values(ascending=False).head(30)

artist
Rick Astley              98300
Luis Fonsi               16180
Queen                     9129
The Lonely Island         6750
Kanye West                6497
Eminem                    5260
"Weird Al" Yankovic       4589
Muse                      4431
Daft Punk                 4413
Radiohead                 4209
Metallica                 4052
Gorillaz                  3506
Pink Floyd                3440
Lemon Demon               3258
Simon & Garfunkel         3202
Childish Gambino          3176
Weezer                    3145
Tame Impala               2995
Red Hot Chili Peppers     2926
Girls' Generation         2878
Foo Fighters              2872
Linkin Park               2822
Michael Jackson           2798
Sabaton                   2769
Rebecca Black             2750
Quindon Tarver            2741
Coldplay                  2738
Europe                    2685
Johnny Cash               2644
Kendrick Lamar            2636
Name: artist, dtype: int64

In [9]:
data=pd.read_csv("data\\Artist_Vectors_f.csv")
gens = pd.read_csv("data\\genre_names_f.csv")

In [10]:
data = pd.merge(data,gens, on ='label')

In [12]:
metadata = data[['artist','genre']]
vectors = data[[str(k) for k in range(150)]]

In [21]:
vectors.to_csv("artist-vector.tsv", index = False, sep='\t', header=False)
metadata.to_csv("artist-metadata.tsv", index = False, sep='\t', header=True)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,0.050776,-0.068250,-0.089456,0.117660,0.007854,-0.017186,0.109276,0.183963,-0.024747,0.076735,...,0.069613,0.060775,0.117048,-0.182615,0.124096,-0.071845,0.078085,-0.010750,-0.072608,0.139540
1,0.069340,-0.056618,-0.107327,0.095203,-0.012574,-0.067289,0.024071,0.205742,-0.028734,0.092169,...,0.041754,0.068061,0.090779,-0.163872,0.166194,-0.058238,0.024600,-0.019799,-0.072912,0.121569
2,0.209051,-0.074679,-0.088339,0.085489,0.024265,-0.041761,-0.019194,0.094542,0.009611,0.076837,...,0.032342,0.097955,0.115424,-0.238258,0.177953,0.021229,0.029554,0.143438,0.003705,0.038490
3,0.092876,-0.091077,-0.098960,0.084772,-0.030727,0.003542,0.088564,0.164386,-0.042881,0.073090,...,0.074165,0.049326,0.098866,-0.157467,0.113963,-0.061070,0.051889,0.002511,-0.078274,0.149645
4,0.104949,-0.082590,-0.125739,0.014587,0.032505,0.006284,0.023797,0.201215,-0.009044,0.094322,...,0.077836,0.107428,0.138836,-0.191603,0.182323,-0.135203,-0.045467,0.074458,-0.088624,0.200642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,0.350018,0.000414,-0.186265,0.144508,0.125924,0.049320,0.723974,0.446921,0.076417,-0.104609,...,0.148358,-0.381416,0.519642,-0.076274,0.205902,-0.370873,0.242421,-0.378226,-0.219339,0.276584
6596,0.340948,0.032373,-0.159011,0.156367,0.140692,0.054098,0.691377,0.362582,0.052454,-0.069452,...,0.186392,-0.351829,0.516834,-0.006048,0.188224,-0.256578,0.259890,-0.317617,-0.282548,0.249341
6597,0.294438,0.087914,-0.330535,0.015182,0.267241,0.126622,0.708479,0.492465,0.033129,-0.256702,...,0.187716,-0.372549,0.385428,-0.342394,0.237522,-0.352479,0.290882,-0.220846,-0.075666,0.207476
6598,0.380099,-0.138049,-0.054309,0.285677,0.108296,0.162640,0.435078,0.227475,-0.093465,-0.017176,...,0.109128,-0.307968,0.426128,-0.125291,0.213913,-0.202980,0.139303,-0.282826,-0.270413,0.196881


In [50]:
artist = 'N.E.R.D'
metadata[metadata['artist']==artist]

,artist,genre
4550,N.E.R.D,rap


In [46]:
artist = 'DIIV'
metadata[metadata['artist']==artist]

,artist,genre
5082,DIIV,indie2
